<a href="https://colab.research.google.com/github/tarakakiranmayi/DeepLearning/blob/main/viratvsdhoni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#virat vs dhoni
import tensorflow as ts
from tensorflow import keras
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from keras.models import Sequential
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [40]:
!mkdir imageRec
cd imageRec
!mkdir train
cd train
!mkdir virat
!mkdir dhoni
cd ..
!mkdir test
cd test
!mkdir virat
!mkdir dhoni
cd ..


mkdir: cannot create directory ‘dhoni’: File exists


In [41]:
!pip install google-api-python-client pillow


In [22]:
import os

# Set the Google Drive directory path


# Set the Google Drive directory path
drive_directory = '/content/drive/imageRec/'

# Create the directory if it doesn’t exist
os.makedirs(drive_directory, exist_ok=True)


# Create the directory if it doesn’t exist



OSError: [Errno 95] Operation not supported: '/content/drive/imageRec/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [74]:
import requests
from googleapiclient.discovery import build
from PIL import Image
from io import BytesIO

# API key and CSE ID
api_key = ''
cse_id = ''

# Function to search for images using Google Custom Search API
def search_images(query, num_results=5):
    service = build("customsearch", "v1", developerKey=api_key)
    result = service.cse().list(q=query, cx=cse_id, searchType='image', num=num_results).execute()
    image_urls = [item['link'] for item in result.get('items', [])]
    return image_urls

# Function to download images
def download_images(image_urls, save_dir='/content/imageRec/test/virat'):
    import os
    os.makedirs(save_dir, exist_ok=True)
    i=13
    for  w,url in enumerate(image_urls):
        try:
            # Request the image
            response = requests.get(url)
            image = Image.open(BytesIO(response.content))

            # Save the image
            image.save(f"{save_dir}/image_{i + 1}.jpg")
            print(f"Downloaded image_{i + 1}.jpg")
        except Exception as e:
            print(f"Could not download image {i + 1}: {e}")
        i+=1

# Usage example
query = "Virat Kohli"
image_urls = search_images(query, num_results=1)
download_images(image_urls)


Downloaded image_14.jpg


In [76]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

# Path to the directory containing the images
image_directory = '/content/imageRec/test/dhoni'

# Initialize the ImageDataGenerator with desired augmentations
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
co=0
# Process all images in the directory
for filename in os.listdir(image_directory):
    # print( filename)
    if filename.endswith(('.png', '.jpg', '.jpeg')) :  # Check for valid image formats
        image_path = os.path.join(image_directory, filename)
        # print(image_path)
        # Load the image
        try:
            # Load the image
            img = load_img(image_path)
            x = img_to_array(img)  # Convert the image to a numpy array
            x = np.expand_dims(x, axis=0)  # Reshape to (1, height, width, channels)

            # Generate augmented images and save them in the same directory
            i=0
            co+=1
            # for batch in datagen.flow(x, batch_size=1, save_to_dir=image_directory,
            #                           save_prefix='aug', save_format='jpeg'):
            #     i += 1
            #     if i >= 4:  # Generate 8 augmented images
            #         break




        except Exception as e:
            print(f"Error processing {filename}: {e}")  # Print error message for debugging

print("Data augmentation completed!",co)


Data augmentation completed! 40


In [105]:
train_ds = keras.utils.image_dataset_from_directory(
    '/content/imageRec/train',
    labels="inferred",
    label_mode="int",  # Use 'int' for sparse labels (0 and 1 for two classes)
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=42,  # For reproducibility
    validation_split=0.2,  # Optional: Split 20% of training data for validation
    subset='training',  # Specify this subset for training
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

val_ds = keras.utils.image_dataset_from_directory(
    '/content/imageRec/train',
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=0.2,  # Use the same split for validation
    subset='validation',  # Specify this subset for validation
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

test_ds = keras.utils.image_dataset_from_directory(
    '/content/imageRec/test',
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=False,  # Typically, you don't shuffle test data
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

Found 215 files belonging to 2 classes.
Using 172 files for training.
Found 215 files belonging to 2 classes.
Using 43 files for validation.
Found 80 files belonging to 2 classes.


In [100]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam

In [106]:
# Load the VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False  # Freeze all layers for initial training

# Create a new model
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Change 2 to the number of your classes
])


In [107]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,  # You can adjust the learning rate
              loss='binary_crossentropy',  # Use 'binary_crossentropy' for binary classification
              metrics=['accuracy'])


In [108]:
train_batches = len(train_ds)
validation_batches = len(test_ds)

# Apply .repeat() to train_ds if len(train_ds) * epochs < steps_per_epoch * epochs
# we need enough data for the training process.
if len(train_ds) * 10 < train_batches * 10:
  train_ds = train_ds.repeat()

In [109]:
history = model.fit(
    train_ds,
    validation_data=val_ds,  # Use validation dataset
    epochs=10
)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 139s 24s/step - accuracy: 0.6324 - loss: 15.2227 - val_accuracy: 0.8837 - val_loss: 1.9959
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 122s 21s/step - accuracy: 0.9501 - loss: 1.1741 - val_accuracy: 0.9535 - val_loss: 1.6672
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 141s 21s/step - accuracy: 0.9963 - loss: 0.2773 - val_accuracy: 0.9767 - val_loss: 2.6428
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 156s 24s/step - accuracy: 1.0000 - loss: 9.0136e-08 - val_accuracy: 0.9767 - val_loss: 3.0677
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 128s 21s/step - accuracy: 0.9881 - loss: 0.4897 - val_accuracy: 0.9767 - val_loss: 2.0922
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 155s 24s/step - accuracy: 1.0000 - loss: 3.4863e-18 - val_accuracy: 0.9767 - val_loss: 1.3959
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 128s 21s/step - accuracy: 0.9881 - loss: 0.5409 - val_accuracy: 0.9767 - val_loss: 1.7721
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 138s 20s/step - accuracy: 1.0000 - loss: 1.3341e-18 - val_accuracy: 0.976

In [112]:
img=image.load_img('/content/imageRec/test/dhoni/image_1.jpg',target_size=(224,224))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array,axis=0)
img_array=img_array/255.0

In [113]:
a=model.predict(img_array)
print(a)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[0.00906513]]
